# Download images from the auction results

Use the url from the auction result to download all images from the auction.

In [ ]:
import os
# setting path
os.chdir(r'..')

import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
SKIPSAVE = cfg['SKIPSAVE']
if not OPBOD:
    month_counter = cfg['URL'][-2:]
else:
    month_counter = cfg['URL'][-4:-2]

if VERBOSE > 0:
    display(cfg)
    
if SKIPSAVE:
    raise InterruptedError('In settings saving is disabled.')



In [2]:
import pandas as pd
import os


def download_images_from_lot(lot, path='../data/images', skip=True):
    '''
    Download .jpg to disk
    '''
    import urllib.request
    import re
    
    urls = lot.Images

    fn_patt = '{}/{}-{}.jpg'.format(path,lot.name,'{:02g}')

    for i,url in enumerate(urls):
        # change url to get larger image
        url = re.sub('1024/768','1024/1024',url)
        # apply format: add counter
        fn = fn_patt.format(i)
        if os.path.isfile(fn) & skip:
            continue
            
        print(url, '->', fn,end='',flush=True)
        urllib.request.urlretrieve(url, fn)
        print('',end='\r')


# Load auction results

(Can also use rdw data here.)

In [ ]:
file_name = f'../data/auctions/results/drz-data-{DATE}-{month_counter}.pkl'
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if not os.path.isfile(file_name):
    # see if -unparsed- exist
    file_name = file_name.replace('auctions/results', 'auctions/temp-results')
    file_name = file_name.replace('drz-data-', 'drz-data-unparsed-')
    if not os.path.isfile(file_name):
        # see if -unparsed without price- exists
        file_name = file_name.replace('auctions/temp-results', 'auctions/without-price')
        if OPBOD:
            file_name = file_name.replace('-opbod.pkl', '-without-price-opbod.pkl')
        else:
            file_name = file_name.replace('.pkl', '-without-price.pkl')

        if not os.path.isfile(file_name):
            # see if -without price- exists (but not unparsed)
            file_name = file_name.replace('drz-data-unparsed-', 'drz-data-')

print(file_name)
drz = pd.read_pickle(file_name)
print(drz.N_images.sum(), 'pictures')

# Download

In [ ]:
if OPBOD:
    images_dir = r'/home/tom/data/satdatsci-images/opbod/' + DATE.replace('-', os.path.sep)
else:
    images_dir = r'/home/tom/data/satdatsci-images/' + DATE.replace('-', os.path.sep)
    
if not os.path.exists(images_dir):
    print(f'Dir does not exist. Create [{images_dir}]')
    os.makedirs(images_dir)
    
# start_idx = pd.np.where(drz.index == '2019-5-8332')[0][0]
start_idx = 0
drz.iloc[start_idx:,:].apply(
    lambda lot:download_images_from_lot(lot, path=images_dir)
    ,axis='columns')

# Next: upload to cloud backup

When images are stored locally it is advised to back them up in the cloud.